In [ ]:
import numpy as np
np.random.seed(1)
from keras.datasets import imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data()
amount_of_train_words = 1000
y_train = y_train.reshape((len(y_test), 1))
y_test = y_test.reshape((len(y_test), 1))
words_indexes = imdb.get_word_index()
rev_words_indexes = dict()
for word, ind in words_indexes.items():
    rev_words_indexes[ind] = word
    
amount_of_examples = len(x_train) #25000
input_feedback = np.zeros((amount_of_examples, amount_of_train_words))
for ex in range(amount_of_examples):
    for ind in x_train[ex]:
        if ind < amount_of_train_words:
            input_feedback[ex][ind] = 1
            

test_feedback = np.zeros((len(x_test), amount_of_train_words))
for ex in range(len(x_test)):
    for ind in x_test[ex]:
        if ind < amount_of_train_words:
            test_feedback[ex][ind] = 1


            
alpha, iterations, hidden_size = (20, 300, 100)
batch_size = 500
weights_0_1 = 0.02 * np.random.random((amount_of_train_words, hidden_size)) - 0.01
weights_1_2 = 0.2 * np.random.random((hidden_size, 1)) - 0.1

relu = lambda x: (x > 0) * x
relu2deriv = lambda x: (x > 0)
sigmoid = lambda x: 1 / (1 + np.exp(-x))
for j in range(iterations):
    correct_cnt, error = 0, 0.0
    for i in range(len(input_feedback) // batch_size):
        batch_st, batch_end = (i * batch_size, (i + 1) * batch_size)
        layer_0 = input_feedback[batch_st: batch_end]
        layer_1 = relu(layer_0.dot(weights_0_1))
        dropout_mask = np.random.randint(2, size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        layer_2 = layer_1.dot(weights_1_2)
        
#         print(y_train[batch_st: batch_end].shape, len(input_feedback) // batch_size)
        for k in range(batch_size):
#             print(layer_2[k: k + 1], y_train[batch_st + k: batch_st + k + 1])
            correct_cnt += int(abs(layer_2[k: k + 1] - y_train[batch_st + k: batch_st + k + 1]) < 0.5)
        
        
        layer_2_delta = (y_train[batch_st: batch_end] - layer_2) / (batch_size * layer_2.shape[0]) # !!!!!!!!!
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * relu2deriv(layer_1)
        layer_1_delta *= dropout_mask
        error += np.sum(layer_2_delta**2, axis=0)
        
        weights_1_2 += alpha * layer_1.T.dot(layer_2_delta) # !!!!!!!!!!
        weights_0_1 += alpha * layer_0.T.dot(layer_1_delta) # !!!!!!!!!!
#         print('@ ', i)
    test_correct_cnt = 0
    for i in range(len(test_feedback)):
        layer_0 = test_feedback[i: i + 1]
        layer_1 = relu(layer_0.dot(weights_0_1))
        layer_2 = layer_1.dot(weights_1_2)
        test_correct_cnt += int(abs(layer_2 - y_test[i: i + 1]) < 0.5)
    print('I:\t', j)
    print('Train-Acc:\t', correct_cnt / float(len(input_feedback)))
    print('Test-Acc:\t', test_correct_cnt / len(test_feedback))
        
        
            


I:	 0
Train-Acc:	 0.50916
Test-Acc:	 0.54948
I:	 1
Train-Acc:	 0.59252
Test-Acc:	 0.67268
I:	 2
Train-Acc:	 0.673
Test-Acc:	 0.75132
I:	 3
Train-Acc:	 0.72876
Test-Acc:	 0.78044
I:	 4
Train-Acc:	 0.76084
Test-Acc:	 0.79452
I:	 5
Train-Acc:	 0.77664
Test-Acc:	 0.80452
I:	 6
Train-Acc:	 0.79164
Test-Acc:	 0.81336
I:	 7
Train-Acc:	 0.79924
Test-Acc:	 0.81936
I:	 8
Train-Acc:	 0.80556
Test-Acc:	 0.82328
I:	 9
Train-Acc:	 0.8116
Test-Acc:	 0.82848
I:	 10
Train-Acc:	 0.81376
Test-Acc:	 0.83148
I:	 11
Train-Acc:	 0.8196
Test-Acc:	 0.83428
I:	 12
Train-Acc:	 0.82368
Test-Acc:	 0.8364
I:	 13
Train-Acc:	 0.82656
Test-Acc:	 0.83864
I:	 14
Train-Acc:	 0.8256
Test-Acc:	 0.84092
I:	 15
Train-Acc:	 0.83168
Test-Acc:	 0.84288
I:	 16
Train-Acc:	 0.82848
Test-Acc:	 0.8434
I:	 17
Train-Acc:	 0.8312
Test-Acc:	 0.84412
I:	 18
Train-Acc:	 0.83392
Test-Acc:	 0.84568
I:	 19
Train-Acc:	 0.83808
Test-Acc:	 0.84632
I:	 20
Train-Acc:	 0.83656
Test-Acc:	 0.84672
I:	 21
Train-Acc:	 0.84004
Test-Acc:	 0.84768
I:	 22

In [56]:
sentence = 'That was really interesting I have read it many times and every minute during watching was astonishing'
l = []
for unit in sentence.lower().split():
    if unit in words_indexes.keys():
        l.append(words_indexes[unit])
    else:
        l.append(0)
        
# l = x_test[4]
hlayer_0 = np.zeros((1, amount_of_train_words))
for ind in l:
    if ind < amount_of_train_words:
        hlayer_0[0][ind] = 1

# print(hlayer_0)
hlayer_1 = relu(hlayer_0.dot(weights_0_1))
hlayer_2 = hlayer_1.dot(weights_1_2)
print(l)
print(hlayer_2)
if hlayer_2 < 0.5:
    print('Negative')
else:
    print('Positive')
    



[12, 13, 63, 218, 10, 25, 329, 9, 108, 208, 2, 172, 783, 312, 146, 13, 5146]
[[0.21275535]]
Negative


In [54]:
print(x_test[1], y_test[1])
for i in x_test[1]:
    print(rev_words_indexes[i], end=' ')

[1, 14, 22, 3443, 6, 176, 7, 5063, 88, 12, 2679, 23, 1310, 5, 109, 943, 4, 114, 9, 55, 606, 5, 111, 7, 4, 139, 193, 273, 23, 4, 172, 270, 11, 7216, 10626, 4, 8463, 2801, 109, 1603, 21, 4, 22, 3861, 8, 6, 1193, 1330, 10, 10, 4, 105, 987, 35, 841, 16873, 19, 861, 1074, 5, 1987, 17975, 45, 55, 221, 15, 670, 5304, 526, 14, 1069, 4, 405, 5, 2438, 7, 27, 85, 108, 131, 4, 5045, 5304, 3884, 405, 9, 3523, 133, 5, 50, 13, 104, 51, 66, 166, 14, 22, 157, 9, 4, 530, 239, 34, 8463, 2801, 45, 407, 31, 7, 41, 3778, 105, 21, 59, 299, 12, 38, 950, 5, 4521, 15, 45, 629, 488, 2733, 127, 6, 52, 292, 17, 4, 6936, 185, 132, 1988, 5304, 1799, 488, 2693, 47, 6, 392, 173, 4, 21686, 4378, 270, 2352, 4, 1500, 7, 4, 65, 55, 73, 11, 346, 14, 20, 9, 6, 976, 2078, 7, 5293, 861, 12746, 5, 4182, 30, 3127, 23651, 56, 4, 841, 5, 990, 692, 8, 4, 1669, 398, 229, 10, 10, 13, 2822, 670, 5304, 14, 9, 31, 7, 27, 111, 108, 15, 2033, 19, 7836, 1429, 875, 551, 14, 22, 9, 1193, 21, 45, 4829, 5, 45, 252, 8, 12508, 6, 565, 921, 3639